# GNSS–Moon Ray Plasmaspheric Integration (PyGCPM)

---

## 1 Environment setup (Colab / Linux)

Mount Drive, install dependencies, clone and compile PyGCPM, copy required data files.

In [ ]:
# ============================================================
# Environment setup (Colab / Linux)
# ============================================================

from google.colab import drive
drive.mount("/content/drive")

# ------------------------------------------------------------
# Python dependencies
# ------------------------------------------------------------
!pip install -q pytecgg astropy jplephem cartopy polars tqdm spiceypy spacepy

# ------------------------------------------------------------
# PyGCPM installation and compilation, and src code
# ------------------------------------------------------------

!git clone -b develop --single-branch https://github.com/viventriglia/iono-lugre.git

# Manual installation of the plasmaspheric model and executable compilation (this is much easier on Linux)
! git clone https://github.com/mattkjames7/PyGCPM
! cp /content/drive/Shareddrives/Iono_LuGRE/GPCM/data/*.dat PyGCPM/PyGCPM/__data/libgcpm/iri/
! pip install -q -e PyGCPM
! cp /content/drive/Shareddrives/Iono_LuGRE/GPCM/data/Kp_ap_since_1932.txt .
%cd /content/PyGCPM/PyGCPM/__data/libgcpm/
! make clean -i > /dev/null
! make > /dev/null
%cd /content/


## 2 Imports, paths, and project modules

Import scientific libraries and project modules.  
Define data, cache, plot, and Kp file paths.


In [ ]:
# ============================================================
# Imports and configuration
# ============================================================

from pathlib import Path
import sys
import os
import urllib.request

import numpy as np
import pandas as pd
import polars as pl
from tqdm.notebook import tqdm

# GNSS / ephemerides
from pytecgg.satellites import prepare_ephemeris, satellite_coordinates
from pytecgg.parsing import read_rinex_nav

from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
import astropy.units as u

from spacepy import coordinates as coord
from spacepy.time import Ticktock

import spiceypy as sp

from PyGCPM import PyGCPM

solar_system_ephemeris.set("jpl")

PROJECT_SRC = Path("/content/iono-lugre/src")
sys.path.insert(0, str(PROJECT_SRC))

from processing_func import process_ray_batch

from utils_func import (
    parse_gnss_arc_id,
    extract_kp_index,
    compute_ray_tangent_points,
)

from plotting_func import plot_integrated_results

# ------------------------------------------------------------
# Paths
# ------------------------------------------------------------

DATA_PATH = Path("/content/drive/Shareddrives/Iono_LuGRE/GPCM/data/")
CACHE_PATH = Path("/content/drive/Shareddrives/Iono_LuGRE/GPCM/cache_runs/")
PLT_PATH = Path("/content/drive/Shareddrives/Iono_LuGRE/Plots_GFLC/GCPM_Plots")
PLT_UTILS_PATH = Path("/content/drive/Shareddrives/Iono_LuGRE/Plots_GFLC")

KP_FILE_PATH = Path("/content/Kp_ap_since_1932.txt")

# ------------------------------------------------------------
# SPICE kernel setup (Moon + Earth orientation)
# ------------------------------------------------------------

KERNEL_PATH = Path("/content/kernels")
KERNEL_PATH.mkdir(parents=True, exist_ok=True)

kernel_urls = {
    "naif0012.tls": "https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls",
    "de440.bsp": "https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de440.bsp",
    "pck00011.tpc": "https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/pck00011.tpc",
    "earth_latest_high_prec.bpc": "https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/earth_latest_high_prec.bpc",
}

for fname, url in kernel_urls.items():
    fpath = KERNEL_PATH / fname
    if not fpath.exists():
        urllib.request.urlretrieve(url, fpath)

sp.furnsh(str(KERNEL_PATH / "naif0012.tls"))
sp.furnsh(str(KERNEL_PATH / "de440.bsp"))
sp.furnsh(str(KERNEL_PATH / "pck00011.tpc"))
sp.furnsh(str(KERNEL_PATH / "earth_latest_high_prec.bpc"))

# sanity check
assert sp.ktotal("ALL") == 4, "SPICE kernels not loaded correctly"


## 3 Arc selection and time grid

Select the GNSS arc of interest, parse satellite/constellation, load observations,  
define the uniform time grid, and list required navigation files.

In [ ]:
# ============================================================
# Arc selection
# ============================================================

arc_of_interest = "G30_OP74_102"
satellite_id, constellation = parse_gnss_arc_id(arc_of_interest)

# Load observation table
obs_tab = pd.read_csv(PLT_UTILS_PATH / "obs_tab_new.csv")

selection_df = obs_tab.loc[
    obs_tab["ArcID"] == arc_of_interest
].copy()
selection_df["time"] = pd.to_datetime(selection_df["time"], errors="raise")

unique_doy = selection_df["time"].dt.dayofyear.unique()

# Uniform time grid (30 s)
times = pl.Series(
    pd.date_range(
        start=selection_df["time"].min(),
        end=selection_df["time"].max(),
        freq="30s"
    ).to_list()
)

# Navigation files
nav_files = [
    DATA_PATH / f"BRDC00IGS_R_2025{doy:03d}0000_01D_MN.rnx"
    for doy in unique_doy
]


## 4 Satellite coordinates from broadcast ephemeris

Compute satellite Cartesian coordinates along the arc using RINEX broadcast ephemerides.

In [ ]:
# ============================================================
# Satellite coordinates from broadcast ephemeris
# ============================================================

sat_x_series, sat_y_series, sat_z_series = [], [], []

for nav_file in nav_files:
    nav_dict = read_rinex_nav(str(nav_file))
    ephemeris_dict = prepare_ephemeris(
        nav_dict, constellation=constellation
    )

    satellite_coords = satellite_coordinates(
        ephem_dict=ephemeris_dict,
        sv_ids=pl.Series([satellite_id] * len(times)),
        gnss_system=constellation,
        epochs=times,
    )

    sat_x_series.append(np.asarray(satellite_coords["sat_x"]))
    sat_y_series.append(np.asarray(satellite_coords["sat_y"]))
    sat_z_series.append(np.asarray(satellite_coords["sat_z"]))

satellite_x = np.concatenate(sat_x_series)
satellite_y = np.concatenate(sat_y_series)
satellite_z = np.concatenate(sat_z_series)


## 5 Lander position in Earth-fixed frame (ITRS)

Compute Lander coordinates in the Earth-fixed (ITRS) frame for the same epochs.


In [ ]:
# ============================================================
# Lunar surface point (LLA) -> ECEF (ITRF93)
# ============================================================

# Time handling
times_astropy = Time(times.to_list(), scale="utc")
ets = np.array([sp.utc2et(t.isot) for t in times_astropy])

# ------------------------------------------------------------
# YOUR lunar coordinates (selenodetic)
# ------------------------------------------------------------
lon_moon = 61.80998    # degrees
lat_moon = 18.56213    # degrees
alt_moon = -3.64896     # km

# Moon mean radius
R_moon = 1737.4  # km
r = R_moon + alt_moon

# Single Moon-fixed Cartesian vector
x_m = r * np.cos(np.deg2rad(lat_moon)) * np.cos(np.deg2rad(lon_moon))
y_m = r * np.cos(np.deg2rad(lat_moon)) * np.sin(np.deg2rad(lon_moon))
z_m = r * np.sin(np.deg2rad(lat_moon))

moon_fixed = np.array([x_m, y_m, z_m])   

# ------------------------------------------------------------
# Moon-fixed -> ECEF (ITRF93)
# ------------------------------------------------------------
lander_pos_ecef = np.zeros((len(ets), 3))

for i, et in enumerate(ets):

    R = sp.pxform("IAU_MOON", "ITRF93", et)
    r_moon_center, _ = sp.spkpos("MOON", et, "ITRF93", "NONE", "EARTH")

    lander_pos_ecef[i] = r_moon_center + R @ moon_fixed

# lander_pos_ecef is now [km] in ECEF (ITRF93)


## 6 Ray construction (satellite → Moon)

Build ray segments connecting satellite positions to Moon positions in Cartesian km.

In [ ]:
# ============================================================
# Build ray segments (satellite → lunar surface point)
# ============================================================

ray_segments = []

for i in range(len(times_astropy)):
    ray_start = [
        satellite_x[i] / 1e3,
        satellite_y[i] / 1e3,
        satellite_z[i] / 1e3,
    ]  # km

    ray_end = [
        lander_pos_ecef[i, 0],
        lander_pos_ecef[i, 1],
        lander_pos_ecef[i, 2],
    ]  # km

    ray_segments.append((ray_start, ray_end))



## 7 Segment TEC analysis (cached)

Run or load cached plasmaspheric TEC integration through the spherical shell.

In [ ]:
# ============================================================
# Segment TEC analysis (cached)
# ============================================================

EARTH_RADIUS_KM = 6371.0
R_inner_km = EARTH_RADIUS_KM
R_outer_km = EARTH_RADIUS_KM + 40000.0

CACHE_PATH.mkdir(parents=True, exist_ok=True)
result_path = CACHE_PATH / f"results_{arc_of_interest}.npy"

if result_path.exists():
    results = np.load(result_path, allow_pickle=True)
else:
    results = process_ray_batch(
        ray_segments=ray_segments,
        times=times.to_list(),
        R_inner_km=R_inner_km,
        R_outer_km=R_outer_km,
        earth_radius_km=EARTH_RADIUS_KM,
        kp_extractor=lambda t: extract_kp_index(t, KP_FILE_PATH),
        PyGCPM=PyGCPM,
        Coords=coord.Coords,
        Ticktock=Ticktock,
        tqdm=tqdm,
        integration_mode='piecewise',
    )
    np.save(result_path, results)


## 8 Tangent point computation and metadata preparation

Compute tangent points and heights and prepare LT/UT metadata for plotting.

In [ ]:
# ============================================================
# Tangent point computation (GNSS satellite → lunar lander)
# ============================================================

tangent_points, tangent_altitudes = compute_ray_tangent_points(
    ray_segments,
    earth_radius_km=EARTH_RADIUS_KM,
)


## 9 Integrated results visualization

Visualize integrated results using the plotting utilities.

In [ ]:
plot_integrated_results(
    results=results,
    times=times.to_list(),
    selection_df=selection_df,
    ray_segments=ray_segments,
    tangent_points=tangent_points,
    tangent_altitudes=tangent_altitudes,
    title=arc_of_interest,
    show_colorbar=True,
    left_letter="g",
    right_letter="h",
    save=False,
    font_scale=1.5,
    n_geodetics=20,
    label_every=3,
    legend_loc="center left",
    max_plot_alt_km=3500.0,
)
